In [69]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar

bld_fn='../data/UCSD_raw_data/max_intersection_BLD_ALL_temp.csv'
pv_fn='../data/UCSD_raw_data/max_intersection_PV_ALL_temp.csv'
weather_fn='../data/UCSD_raw_data/Weather_SanDiego.csv'

bld=pd.read_csv(bld_fn)
bld['DateTime']=pd.to_datetime(bld["DateTime"])
pv=pd.read_csv(pv_fn)
pv['DateTime']=pd.to_datetime(pv["DateTime"])

In [2]:
bld=bld.drop_duplicates()

In [70]:
#read weather data and transform the datetime
weather=pd.read_csv(weather_fn)
weather['DateTime'] = pd.to_datetime(\
    weather['dt_iso'].apply(lambda x: x.replace(' +0000 UTC', '')), format='%Y-%m-%d %H:%M:%S')

#drop unnecessary columns
weather=weather.drop(columns=['dt_iso','timezone','city_name',
                              'lat','lon','sea_level','grnd_level',
                              'weather_id','weather_icon','weather_description'])

#replace weather description with intergers
column_dict = {'Clear': 1, 'Clouds':2, "Drizzle":3, "Dust":4,"Fog":5, 
               "Haze":6, "Mist":7, "Rain":8, "Smoke":9, "Thunderstorm":10}
weather = weather.replace({"weather_main": column_dict})
weather['weather_main'].astype(int)

#helper function for encoding historical data
def encode_shift(encode,df,unit):
        len(encode['keys'])
        df_encode=df.copy()
        #add a column for further merge
        df_encode['hour_match']=df_encode['DateTime'].dt.strftime("%Y-%m-%d-%H")
        for i in range(len(encode["keys"])):
                for k in range(encode["shift"][i-1]):
                        df_encode[encode["keys"][i-1]+'_-'+str(k+1)+unit]=df[encode["keys"][i-1]].shift(k+1)
        return df_encode

#def encode_load_shift()

#helper function for encoding historical data
def encode_time(df):
        cal = USFederalHolidayCalendar()
        holidays = cal.holidays(start='2014-01-01', end='2021-12-31').to_pydatetime()
        df['hour'] = df['DateTime'].dt.hour
        df['dayofweek'] = df['DateTime'].dt.dayofweek
        df['quarter'] = df['DateTime'].dt.quarter
        df['month'] = df['DateTime'].dt.month
        df['year'] = df['DateTime'].dt.year
        df['dayofyear'] = df['DateTime'].dt.dayofyear
        df['dayofmonth'] = df['DateTime'].dt.day
        #df['weekofyear'] = df['DateTime'].dt.weekofyear
        df['is_holiday'] = df['DateTime'].apply(lambda x: x in holidays)
        return df

In [71]:
weather.info(
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44320 entries, 0 to 44319
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   temp          44320 non-null  float64       
 1   feels_like    44320 non-null  float64       
 2   temp_min      44320 non-null  float64       
 3   temp_max      44320 non-null  float64       
 4   pressure      44320 non-null  int64         
 5   humidity      44320 non-null  int64         
 6   wind_speed    44320 non-null  float64       
 7   wind_deg      44320 non-null  int64         
 8   rain_1h       3412 non-null   float64       
 9   rain_3h       296 non-null    float64       
 10  snow_1h       0 non-null      float64       
 11  snow_3h       0 non-null      float64       
 12  clouds_all    44320 non-null  int64         
 13  weather_main  44320 non-null  int64         
 14  DateTime      44320 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(9), in

In [72]:
weather.drop_duplicates()

,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_main,DateTime
0,10.83,9.20,10.55,11.03,1017,47,3.1,280,NaN,NaN,NaN,NaN,40,2,2015-01-01 00:00:00
1,10.57,8.70,9.95,11.03,1017,39,2.6,340,NaN,NaN,NaN,NaN,40,2,2015-01-01 01:00:00
2,9.94,9.94,8.53,9.95,1018,32,0.0,0,NaN,NaN,NaN,NaN,20,2,2015-01-01 02:00:00
3,9.06,9.06,6.83,9.35,1018,33,0.0,0,NaN,NaN,NaN,NaN,20,2,2015-01-01 03:00:00
4,8.03,8.03,5.13,8.23,1018,38,0.0,0,NaN,NaN,NaN,NaN,20,2,2015-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44315,17.68,16.65,16.59,19.69,1018,44,2.6,220,NaN,NaN,NaN,NaN,75,2,2020-01-01 19:00:00
44316,17.98,17.24,16.59,19.69,1016,54,3.6,180,NaN,NaN,NaN,NaN,75,2,2020-01-01 20:00:00
44317,17.86,17.40,16.02,19.97,1015,65,3.6,220,NaN,NaN,NaN,NaN,75,2,2020-01-01 21:00:00
44318,17.78,17.47,16.02,19.46,1015,71,2.6,230,NaN,NaN,NaN,NaN,75,2,2020-01-01 22:00:00


In [73]:
helper=pd.DataFrame({'DateTime':pd.date_range(start=weather['DateTime'].min(),
                                                end=weather['DateTime'].max(),freq='1h')})
weather=pd.merge(helper,weather,on=['DateTime'],how='left').sort_values("DateTime")
helper
#weather[key]=weather[key].interpolate(method ='linear')

,DateTime
0,2015-01-01 00:00:00
1,2015-01-01 01:00:00
2,2015-01-01 02:00:00
3,2015-01-01 03:00:00
4,2015-01-01 04:00:00
...,...
43843,2020-01-01 19:00:00
43844,2020-01-01 20:00:00
43845,2020-01-01 21:00:00
43846,2020-01-01 22:00:00


In [76]:
weather=weather.drop_duplicates(subset='DateTime')

In [77]:
weather

,DateTime,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_main
0,2015-01-01 00:00:00,10.83,9.20,10.55,11.03,1017,47,3.1,280,NaN,NaN,NaN,NaN,40,2
1,2015-01-01 01:00:00,10.57,8.70,9.95,11.03,1017,39,2.6,340,NaN,NaN,NaN,NaN,40,2
2,2015-01-01 02:00:00,9.94,9.94,8.53,9.95,1018,32,0.0,0,NaN,NaN,NaN,NaN,20,2
3,2015-01-01 03:00:00,9.06,9.06,6.83,9.35,1018,33,0.0,0,NaN,NaN,NaN,NaN,20,2
4,2015-01-01 04:00:00,8.03,8.03,5.13,8.23,1018,38,0.0,0,NaN,NaN,NaN,NaN,20,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44315,2020-01-01 19:00:00,17.68,16.65,16.59,19.69,1018,44,2.6,220,NaN,NaN,NaN,NaN,75,2
44316,2020-01-01 20:00:00,17.98,17.24,16.59,19.69,1016,54,3.6,180,NaN,NaN,NaN,NaN,75,2
44317,2020-01-01 21:00:00,17.86,17.40,16.02,19.97,1015,65,3.6,220,NaN,NaN,NaN,NaN,75,2
44318,2020-01-01 22:00:00,17.78,17.47,16.02,19.46,1015,71,2.6,230,NaN,NaN,NaN,NaN,75,2


In [78]:
encode={
    'keys':['temp', 'feels_like', 'temp_min', 
            'temp_max', 'pressure', 'humidity',
            'wind_speed', 'wind_deg', 'rain_1h', 
            'rain_3h', 'snow_1h', 'snow_3h',
            'clouds_all', 'weather_main'],
    'shift':[12,12,3,
             3,3,3,
             3,3,3,
             3,3,3,
             3,3]
}
weather_encode=encode_shift(encode,weather,'h')

In [97]:

encode={
    'keys':['RealPower'],
    'shift':[24]
}
bld=encode_shift(encode,bld,'q')
pv=encode_shift(encode,pv,"q")
bld=encode_time(bld)
pv=encode_time(pv)
bld.convert_dtypes(convert_boolean='int')
pv.convert_dtypes(convert_boolean='int')

,DateTime,RealPower,hour_match,RealPower_-1q,RealPower_-2q,RealPower_-3q,RealPower_-4q,RealPower_-5q,RealPower_-6q,RealPower_-7q,...,RealPower_-23q,RealPower_-24q,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,is_holiday
0,2015-01-01 00:00:00,0.02,2015-01-01-00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,0,3,1,1,2015,1,1,True
1,2015-01-01 00:15:00,0.019,2015-01-01-00,0.02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,0,3,1,1,2015,1,1,False
2,2015-01-01 00:30:00,0.018,2015-01-01-00,0.019,0.02,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,0,3,1,1,2015,1,1,False
3,2015-01-01 00:45:00,0.016,2015-01-01-00,0.018,0.019,0.02,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,0,3,1,1,2015,1,1,False
4,2015-01-01 01:00:00,0.018,2015-01-01-01,0.016,0.018,0.019,0.02,<NA>,<NA>,<NA>,...,<NA>,<NA>,1,3,1,1,2015,1,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181051,2020-02-29 22:45:00,0.0,2020-02-29-22,0.0,0.0,-0.001,-0.001,-0.001,-0.001,-0.003,...,29.478,58.855,22,5,1,2,2020,60,29,False
181052,2020-02-29 23:00:00,-0.001,2020-02-29-23,0.0,0.0,0.0,-0.001,-0.001,-0.001,-0.001,...,19.425,29.478,23,5,1,2,2020,60,29,False
181053,2020-02-29 23:15:00,-0.001,2020-02-29-23,-0.001,0.0,0.0,0.0,-0.001,-0.001,-0.001,...,11.141,19.425,23,5,1,2,2020,60,29,False
181054,2020-02-29 23:30:00,-0.001,2020-02-29-23,-0.001,-0.001,0.0,0.0,0.0,-0.001,-0.001,...,4.274,11.141,23,5,1,2,2020,60,29,False


In [98]:
bld['is_holiday']=bld['is_holiday'].astype(int)
pv['is_holiday']=pv['is_holiday'].astype(int)

In [99]:
pv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181056 entries, 0 to 181055
Data columns (total 35 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   DateTime        181056 non-null  datetime64[ns]
 1   RealPower       181056 non-null  float64       
 2   hour_match      181056 non-null  object        
 3   RealPower_-1q   181055 non-null  float64       
 4   RealPower_-2q   181054 non-null  float64       
 5   RealPower_-3q   181053 non-null  float64       
 6   RealPower_-4q   181052 non-null  float64       
 7   RealPower_-5q   181051 non-null  float64       
 8   RealPower_-6q   181050 non-null  float64       
 9   RealPower_-7q   181049 non-null  float64       
 10  RealPower_-8q   181048 non-null  float64       
 11  RealPower_-9q   181047 non-null  float64       
 12  RealPower_-10q  181046 non-null  float64       
 13  RealPower_-11q  181045 non-null  float64       
 14  RealPower_-12q  181044 non-null  flo

In [100]:
weather_encode

,DateTime,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,...,rain_3h_-3h,snow_1h_-1h,snow_1h_-2h,snow_1h_-3h,snow_3h_-1h,snow_3h_-2h,snow_3h_-3h,clouds_all_-1h,clouds_all_-2h,clouds_all_-3h
0,2015-01-01 00:00:00,10.83,9.20,10.55,11.03,1017,47,3.1,280,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01 01:00:00,10.57,8.70,9.95,11.03,1017,39,2.6,340,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN
2,2015-01-01 02:00:00,9.94,9.94,8.53,9.95,1018,32,0.0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,40.0,NaN
3,2015-01-01 03:00:00,9.06,9.06,6.83,9.35,1018,33,0.0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,40.0,40.0
4,2015-01-01 04:00:00,8.03,8.03,5.13,8.23,1018,38,0.0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,20.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44315,2020-01-01 19:00:00,17.68,16.65,16.59,19.69,1018,44,2.6,220,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,75.0,40.0
44316,2020-01-01 20:00:00,17.98,17.24,16.59,19.69,1016,54,3.6,180,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,40.0,75.0
44317,2020-01-01 21:00:00,17.86,17.40,16.02,19.97,1015,65,3.6,220,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,75.0,40.0
44318,2020-01-01 22:00:00,17.78,17.47,16.02,19.46,1015,71,2.6,230,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,75.0,75.0


In [101]:
bld=bld.drop_duplicates(subset="DateTime")

In [102]:
bld

,DateTime,RealPower,hour_match,RealPower_-1q,RealPower_-2q,RealPower_-3q,RealPower_-4q,RealPower_-5q,RealPower_-6q,RealPower_-7q,...,RealPower_-23q,RealPower_-24q,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,is_holiday
0,2015-01-01 00:00:00,180.432,2015-01-01-00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,3,1,1,2015,1,1,1
1,2015-01-01 00:15:00,180.292,2015-01-01-00,180.432,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,3,1,1,2015,1,1,0
2,2015-01-01 00:30:00,179.689,2015-01-01-00,180.292,180.432,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,3,1,1,2015,1,1,0
3,2015-01-01 00:45:00,179.773,2015-01-01-00,179.689,180.292,180.432,NaN,NaN,NaN,NaN,...,NaN,NaN,0,3,1,1,2015,1,1,0
4,2015-01-01 01:00:00,178.092,2015-01-01-01,179.773,179.689,180.292,180.432,NaN,NaN,NaN,...,NaN,NaN,1,3,1,1,2015,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181051,2020-02-29 22:45:00,318.207,2020-02-29-22,321.394,335.710,336.379,327.082,324.638,332.320,342.551,...,357.872,351.210,22,5,1,2,2020,60,29,0
181052,2020-02-29 23:00:00,322.779,2020-02-29-23,318.207,321.394,335.710,336.379,327.082,324.638,332.320,...,358.494,357.872,23,5,1,2,2020,60,29,0
181053,2020-02-29 23:15:00,328.058,2020-02-29-23,322.779,318.207,321.394,335.710,336.379,327.082,324.638,...,359.805,358.494,23,5,1,2,2020,60,29,0
181054,2020-02-29 23:30:00,325.368,2020-02-29-23,328.058,322.779,318.207,321.394,335.710,336.379,327.082,...,378.665,359.805,23,5,1,2,2020,60,29,0


In [103]:
pv=pv.drop_duplicates(subset="DateTime")

In [104]:
xgboost_data_bld=pd.merge(bld,weather_encode,how='left',on='hour_match')
xgboost_data_bld=xgboost_data_bld.drop(columns=['hour_match','DateTime_y'])
xgboost_data_bld=xgboost_data_bld.drop_duplicates(subset="DateTime_x")
xgboost_data_bld=xgboost_data_bld.set_index('DateTime_x')

xgboost_data_pv=pd.merge(pv,weather_encode,how='left',on='hour_match')
xgboost_data_pv=xgboost_data_pv.drop(columns=['hour_match','DateTime_y'])
xgboost_data_pv=xgboost_data_pv.drop_duplicates(subset="DateTime_x")
xgboost_data_pv=xgboost_data_pv.set_index('DateTime_x')

In [105]:
xgboost_data_pv.index.name='DateTime'
xgboost_data_bld.index.name='DateTime'

In [106]:
xgboost_data_pv

,RealPower,RealPower_-1q,RealPower_-2q,RealPower_-3q,RealPower_-4q,RealPower_-5q,RealPower_-6q,RealPower_-7q,RealPower_-8q,RealPower_-9q,...,rain_3h_-3h,snow_1h_-1h,snow_1h_-2h,snow_1h_-3h,snow_3h_-1h,snow_3h_-2h,snow_3h_-3h,clouds_all_-1h,clouds_all_-2h,clouds_all_-3h
DateTime,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,0.020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:15:00,0.019,0.020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:30:00,0.018,0.019,0.020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 00:45:00,0.016,0.018,0.019,0.020,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 01:00:00,0.018,0.016,0.018,0.019,0.020,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29 22:45:00,0.000,0.000,0.000,-0.001,-0.001,-0.001,-0.001,-0.003,-0.003,-0.004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-29 23:00:00,-0.001,0.000,0.000,0.000,-0.001,-0.001,-0.001,-0.001,-0.003,-0.003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-29 23:15:00,-0.001,-0.001,0.000,0.000,0.000,-0.001,-0.001,-0.001,-0.001,-0.003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
xgboost_data_pv.to_csv('../data/XGBoost_toy_case/merged_pv_new.csv')
xgboost_data_bld.to_csv('../data/XGBoost_toy_case/merged_bld_new.csv')

In [108]:
xgboost_data_bld.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 182944 entries, 2015-01-01 00:00:00 to 2020-02-29 23:45:00
Columns: 107 entries, RealPower to clouds_all_-3h
dtypes: Float64(25), Int32(7), float64(74), int32(1)
memory usage: 150.7 MB
